In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path

sns.set_context("talk"); sns.set_style("whitegrid")
signals_path = Path("../data/trends_signals.csv")
df = pd.read_csv(signals_path, parse_dates=["date"])
df["month"] = df["date"].dt.to_period("M").astype(str)
df.head()


In [ ]:
CRIT_YOY = 110.0
CRIT_Z   = 1.2

agg = (df.groupby(["keyword","month"], as_index=False)
         .agg(avg_trend=("trend_ma","mean"),
              avg_yoy=("yoy_idx","mean"),
              avg_z=("z_score","mean"),
              days=("date","count"),
              hot_days=("z_score", lambda s: int(((s>=CRIT_Z) & (s.notna())).sum()))))
agg["hot_share"] = agg["hot_days"] / agg["days"]
agg.head(10)


In [ ]:
def minmax(x):
    x = x.astype(float)
    lo, hi = np.nanmin(x), np.nanmax(x)
    return (x - lo) / (hi - lo) if hi > lo else np.zeros_like(x)

tmp = agg.copy()
tmp["z_scaled"]   = minmax(tmp["avg_z"].clip(-3, 5))
tmp["yoy_scaled"] = minmax(tmp["avg_yoy"].clip(80, 200))
tmp["act_score"]  = 0.6*tmp["z_scaled"] + 0.3*tmp["yoy_scaled"] + 0.1*tmp["hot_share"]

top_months = (tmp.sort_values(["keyword","act_score"], ascending=[True, False])
                .groupby("keyword", as_index=False)
                .head(3)[["keyword","month","act_score","avg_yoy","avg_z","hot_share"]])
top_months


In [ ]:
import seaborn as sns
pv = tmp.pivot(index="keyword", columns="month", values="act_score").fillna(0)

plt.figure(figsize=(14, 5 + 0.4*pv.shape[0]))
sns.heatmap(pv, cmap="viridis", linewidths=.3, cbar_kws={"label": "Activation score"})
plt.title("Activation Radar — momentum by month")
plt.xlabel("Month"); plt.ylabel("Keyword")
plt.tight_layout()

out_png = Path("../assets/activation_radar.png")
out_png.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(out_png, dpi=160)
out_png.as_posix()


In [ ]:
from pathlib import Path
excel_path = Path("../reports/sample_activation_radar.xlsx")
excel_path.parent.mkdir(parents=True, exist_ok=True)

with pd.ExcelWriter(excel_path, engine="xlsxwriter", mode="a", if_sheet_exists="replace") as writer:
    tmp.sort_values(["keyword","month"]).to_excel(writer, index=False, sheet_name="Activation_Radar")
    top_months.to_excel(writer, index=False, sheet_name="Top_Months")

    wb = writer.book
    # autosize columns
    for name, df_ in [("Activation_Radar", tmp), ("Top_Months", top_months)]:
        ws = writer.sheets[name]
        for j, col in enumerate(df_.columns):
            width = max(12, min(40,
                                len(str(col))+4,
                                int(df_[col].astype(str).str.len().quantile(0.9))+2))
            ws.set_column(j, j, width)

excel_path.as_posix()


In [ ]:
reco = (top_months.sort_values(["keyword","act_score"], ascending=[True, False])
        .groupby("keyword").head(1).copy())
reco["rec"] = ("Prioritize activation in " + reco["month"] +
               " given high momentum (score=" + reco["act_score"].round(2).astype(str) +
               ", YoY=" + reco["avg_yoy"].round(1).astype(str) +
               ", z=" + reco["avg_z"].round(2).astype(str) + ").")
reco[["keyword","rec"]]
